In [1]:
import sympy as sm

In [2]:
from sympy.physics.vector import init_vprinting
init_vprinting(use_latex="mathjax", pretty_print=False)

In [41]:
from sympy.physics.mechanics import dynamicsymbols

In [42]:
g = sm.symbols('g', real=True)
m1, m2, l1, l2, theta1, theta2 = dynamicsymbols('m1, m2, l1, l2, theta1, theta2')
theta1d = dynamicsymbols('theta1',1)
theta2d = dynamicsymbols('theta2',1)
m1, m2, l1, l2, theta1, theta2, theta1d, theta2d,g

(m1, m2, l1, l2, theta1, theta2, theta1', theta2', g)

In [43]:
# kinetic energy
ke = ((1/2)*(m1 + m2)*(sm.Pow(l1,2))*(sm.Pow(theta1d,2)))
ke += ((1/2)*(m2)*(sm.Pow(l1,2))*(sm.Pow(theta1d,2)))
ke += ((1/2)*(m2)*(sm.Pow(l2,2))*(theta1d)*(theta2d))
ke += ((1/2)*(m2)*(sm.Pow(l2,2))*(sm.Pow(theta2d,2)))
ke += (m2*l1*l2)*sm.cos(theta2)*((theta1d*theta2d)+sm.Pow(theta1d,2))
ke

(0.5*m1 + 0.5*m2)*l1**2*theta1'**2 + (theta1'**2 + theta1'*theta2')*l1*l2*m2*cos(theta2) + 0.5*l1**2*m2*theta1'**2 + 0.5*l2**2*m2*theta1'*theta2' + 0.5*l2**2*m2*theta2'**2

In [36]:
# potential energy
pe = m1*g*l1*sm.cos(theta1)
pe += m2*g*(l1*sm.cos(theta1) +l2*sm.cos(theta1+theta2))
pe

(l1*cos(theta1) + l2*cos(theta1 + theta2))*g*m2 + g*l1*m1*cos(theta1)

In [46]:
# the lagrangian equation
L = ke - pe

(l1*sin(theta1) + l2*sin(theta1 + theta2))*g*m2' + (l1*sin(theta1) + l2*sin(theta1 + theta2))*m2*g' + ((theta1' + theta2')*l2*cos(theta1 + theta2) + l1*cos(theta1)*theta1' + sin(theta1 + theta2)*l2' + sin(theta1)*l1')*g*m2 + g*l1*m1*cos(theta1)*theta1' + g*l1*sin(theta1)*m1' + g*m1*sin(theta1)*l1' + l1*m1*sin(theta1)*g' - theta1'

In [49]:
# force applied at joint 1
f1 = sm.diff(sm.diff(L,theta1),'t') - sm.diff(theta1)
f1sim = sm.simplify(f1)
f1sim

(l1*sin(theta1) + l2*sin(theta1 + theta2))*g*m2' + (l1*sin(theta1) + l2*sin(theta1 + theta2))*m2*g' + ((theta1' + theta2')*l2*cos(theta1 + theta2) + l1*cos(theta1)*theta1' + sin(theta1 + theta2)*l2' + sin(theta1)*l1')*g*m2 + g*l1*m1*cos(theta1)*theta1' + g*l1*sin(theta1)*m1' + g*m1*sin(theta1)*l1' + l1*m1*sin(theta1)*g' - theta1'

In [50]:
# force applied at joint 2
f2 = sm.diff(sm.diff(L,theta2),'t') - sm.diff(theta2)
f2sim = sm.simplify(f2)
f2sim

(theta1' + theta2')*g*l2*m2*cos(theta1 + theta2) - (theta1' + theta2')*l1*l2*m2*cos(theta2)*theta1'*theta2' - (theta1' + theta2')*l1*l2*sin(theta2)*m2'*theta1' - (theta1' + theta2')*l1*m2*sin(theta2)*l2'*theta1' - (theta1' + theta2')*l2*m2*sin(theta2)*l1'*theta1' - (2*theta1'*theta1'' + theta1'*theta2'' + theta1''*theta2')*l1*l2*m2*sin(theta2) + g*l2*sin(theta1 + theta2)*m2' + g*m2*sin(theta1 + theta2)*l2' + l2*m2*sin(theta1 + theta2)*g' - theta2'